<img width="600" height="200" src="../../datasets/redes.png">

# Animations with Earth Engine
## Purpose
The purpose of this notebook is to create animations from Earth Engine datasets.

## Setup
### Library import

In [1]:
import os
from ipywidgets import interact
import matplotlib.pyplot as plt

from vizzDL import animations
from vizzDL.models.CNN.super_resolution import srgan

## **Create animation**
**Create `animations` object**

In [2]:
animation = animations.Animation()

**Select region**

In [3]:
geometry = {'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'properties': {},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-122.235435, 46.00532],
      [-122.235435, 46.213047],
      [-121.982083, 46.213047],
      [-121.982083, 46.00532],
      [-122.235435, 46.00532]]]}}]}

In [4]:
animation.select_region(instrument='Landsat', geometry=geometry, lat=46.056, lon=-122.1149, zoom=10)

Map(center=[46.056, -122.1149], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'z…

### Time-lapse

**Save images as `PNG`s**

In [6]:
animation.save_frames_as_PGNs(folder_path = '../../datasets/processed/GeoTIFFs/', region_name = 'Saint_Helens', start_year=1995, stop_year=2019)

Image number: 0
Image number: 1
Image number: 2
Image number: 3


HTTPError: HTTP Error 400: Bad Request

**Create animation as a Numpy array**

In [6]:
video = animation.video_as_array(start_year=2020, stop_year=2021)

Image number: 0
Image number: 1


### Prediction
**Create model**

In [7]:
# Location of model weights
weights_dir = '../../datasets/processed/Models/L8_S2_SR_x3/srgan_generator_L8_to_S2_x3'
weights_file = lambda filename: os.path.join(weights_dir, filename)

pre_generator = srgan.Generator(input_shape=(None, None, 3), scale=3).generator()
pre_generator.load_weights(weights_file('model_weights.h5'))

**predict**

In [8]:
prediction = animation.predict(model = pre_generator, norm_range=[[0,1], [-1,1]])

ResourceExhaustedError: Graph execution error:

Detected at node 'generator/up_sampling2d/resize/ResizeNearestNeighbor' defined at (most recent call last):
    File "/usr/lib/python3.8/runpy.py", line 194, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib/python3.8/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/usr/local/lib/python3.8/dist-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/usr/local/lib/python3.8/dist-packages/traitlets/config/application.py", line 972, in launch_instance
      app.start()
    File "/usr/local/lib/python3.8/dist-packages/ipykernel/kernelapp.py", line 505, in start
      self.io_loop.start()
    File "/usr/local/lib/python3.8/dist-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib/python3.8/asyncio/base_events.py", line 570, in run_forever
      self._run_once()
    File "/usr/l

In [9]:
@interact(frame=(0, video.shape[0]-1))
def show_frame(frame=0):
    fig, ax = plt.subplots(1, 2, figsize=(20,20))

    ax[0].imshow(video[frame,:,:,:])
    ax[0].set_title('LR')

    ax[1].imshow(prediction[frame,:,:,:])
    ax[1].set_title('SR (PRE)')

interactive(children=(IntSlider(value=0, description='frame', max=1), Output()), _dom_classes=('widget-interac…

### Create tiles
**Create tiles**

In [ ]:
animation.create_animated_tiles(folder_path = '../../datasets/processed/Tiles/', region_name = 'Menongue', minZ = 10, maxZ=14)